In [ ]:
import pandas               as pd
import matplotlib.pyplot    as plt
import numpy                as np
import glob
import random

training_image_path = 'datasets/training/'
training_tnf_csv    = 'training_data/tps'

train_pd = pd.read_csv(training_tnf_csv + '/train.csv') 

In [ ]:
transforms_pd = pd.DataFrame(columns=['Parameter','Min','Max'])

# Find the min & max values of each parameter
for i in range(1,73):
    name = 't' + str(i)
    data = train_pd[name]
    plt.scatter(np.linspace(0, 1, len(data)), data)
    
    transforms_pd = transforms_pd.append({
        'Parameter': name, 
        'Min':       np.amin(data), 
        'Max':       np.amax(data)    
        }, ignore_index=True)
    

In [ ]:
def random_transform(pd='', image_name='', min_max=[], name=True):
    """
    Create a random transform by sampling from each parameter. 
    Assume a uniform distribution between min & max values.
    """
    
    output = {}
    if name:
        output['ImageA'] = image_name
        output['ImageB'] = image_name
        
    if len(min_max):
        for i in range(1,19):
            name    = 't' + str(i)
            value   = np.random.uniform(min_max[0,i-1], min_max[1,i-1])
            output[name] = [value]
            
    else:
        for i in range(1,73):
            name    = 't' + str(i)
            row     = pd[pd['Parameter']==name]
            value   = np.random.uniform(row['Min'], row['Max'])[0]
            output[name] = [value]
            
    
        
    return output

### Add MRI images

In [ ]:
file_names = ['HMU_003_DB', 'HMU_011_MQ', 'HMU_025_SH']

in_vivo = {
    'HMU_003_DB': [20,19,12],
    'HMU_007_TN': [19,18,16],
    'HMU_010_FH' : [19,17,13,12],
    'HMU_011_MQ': [18,12,10,9],
    'HMU_025_SH': [22,19,15]
}

updated_train_pd = train_pd.copy()
number_samples   = 0

for name in file_names:
    # MRI 
    mri_slices = in_vivo[name]
    for slice in mri_slices:
        image_name  = 'mri_' + name + '_' + str(slice) + '.png'
        new_row     = random_transform(transforms_pd, image_name)
        
        new_row_pd       = pd.DataFrame.from_dict(new_row)
        updated_train_pd = pd.concat((updated_train_pd, new_row_pd), ignore_index=True)
        
        number_samples += 1

### Add histology images

In [ ]:
file_names = ['HMU_003_DB', 'HMU_011_MQ', 'HMU_025_SH','HMU_056_JH','HMU_060_CH','HMU_063_RS','HMU_064_SB','HMU_065_RH','HMU_067_MS']

updated_train_pd = train_pd.copy()
number_samples   = 0

histo = {
    'HMU_003_DB': ['_A1', '_A2','_A3','_A5'],
    'HMU_011_MQ': ['_4','_5','_6','_7','_8','_9'],
    'HMU_025_SH': ['A1','A3','A4','A5'],
    'HMU_056_JH': ['A1','A2','A3','A4','A5','A6'], 
    'HMU_060_CH': ['A1','A2','A3','A4','A5','A6'], 
    'HMU_063_RS': ['A1','A2','A3','A4','A5'], 
    'HMU_064_SB': ['A1','A4','A6'], 
    'HMU_065_RH': ['A1','A2','A3','A4','A5','A6'], 
    'HMU_067_MS': ['A1','A2','A3','A4','A5','A6','A7']
}

for i in range(10):
    for name in file_names: 
        histo_slices = histo[name]
        for slice in histo_slices:
            image_name  = name + str(slice) + '_segmented.png'
            new_row     = random_transform(transforms_pd, image_name)
            
            new_row_pd       = pd.DataFrame.from_dict(new_row)
            updated_train_pd = pd.concat((updated_train_pd, new_row_pd), ignore_index=True)
            
            number_samples += 1

print(number_samples)
updated_train_pd

updated_train_pd.to_csv('training_data/tps/train_updated_47.csv', index=False)

### Illustrate TPS transforms 

In [ ]:
from process_img import * 
from geotnf.transformation import GeometricTnf
from skimage import io


def tps_transform_grid(transforms_pd):
    """
    Apply a random TPS transform to the sample grid
    """
    
    transform = random_transform(transforms_pd, name=False)
    theta_tps = [transform[i] for i in transform]
    theta_tps = torch.Tensor(np.transpose(theta_tps))

    # Preprocess image 
    source_image = io.imread('../Dataset/Data/grid.png')
    source_image = np.squeeze(source_image[:,:,:3])
    source_image[source_image<250] = 0
    source_image = process_image(source_image,use_cuda=False, out_size=1024)

    # TPS transformation
    tpsTnf       = GeometricTnf(geometric_model='tps', out_h=400, out_w=400, use_cuda=False)
    warped_image = tpsTnf(source_image,theta_tps)
        
    # Un-normalize images and convert to numpy
    warped_image_np = normalize_image(warped_image,forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
        
    # Ignore negative values
    warped_image_np[warped_image_np < 0] = 0    
    source_image = source_image.permute(0,2,3,1)
    return source_image, warped_image_np

In [ ]:
source_image, warped_image = tps_transform_grid(transforms_pd)

fig, axs = plt.subplots(1,2)
axs[0].imshow(np.squeeze(source_image))
axs[1].imshow(warped_image)

In [ ]:
from process_img import * 
from geotnf.transformation import GeometricTnf
from skimage import io
from torch.nn.functional import normalize

def tps_Grid(theta_tps, norm=False):
    """
    Apply a random TPS transform to the sample grid
    """
    
    theta_tps = torch.Tensor(theta_tps)
    if norm:
        theta_tps = normalize(theta_tps)

    # Preprocess image 
    source_image = io.imread('../Dataset/Data/grid.png')
    source_image = np.squeeze(source_image[:,:,:3])
    source_image[source_image<250] = 0
    source_image = process_image(source_image, use_cuda=False, out_size=1024)

    # TPS transformation
    tpsTnf_mri   = GeometricTnf(geometric_model='tps-mri', out_h=400, out_w=400, use_cuda=False)
    warped_image = tpsTnf_mri(source_image,theta_tps)
        
    # Un-normalize images and convert to numpy
    warped_image_np = normalize_image(warped_image,forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
        
    # Ignore negative values
    warped_image_np[warped_image_np < 0] = 0    
    source_image = source_image.permute(0,2,3,1)
    
    fig, axs = plt.subplots(1,2)
    axs[0].imshow(np.squeeze(source_image))
    axs[1].imshow(warped_image_np)

In [ ]:
tps =[
            [
                -0.6362779140472412,
                -1.6981632709503174,
                0.587742805480957,
                0.8616968393325806,
                -2.4061849117279053,
                -4.045632839202881,
                -0.05718374252319336,
                1.5738213062286377,
                1.9545316696166992,
                -2.3466782569885254,
                1.491770625114441,
                1.8796687126159668,
                -1.5033074617385864,
                -2.048987865447998,
                2.108168601989746,
                0.7781451940536499,
                -0.4112209379673004,
                1.1588855981826782
            ]
        ]
tps_test =[[-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,1.0,1.0,-1.0,0.0,1.0,-1.0,0.0,1.0,-1.0,0.0,1.0]]

tps_Grid(tps_test)
tps_Grid(tps)
tps_Grid(tps, norm=True)

In [ ]:
min_max = [[-1.1, -1.1, -1.1, -0.1, -0.1, -0.1, 0.9, 0.9, 0.9,-1.1, -0.1, 0.9,-1.1, -0.1, 0.9,-1.1, -0.1, 0.9],
            [-0.9, -0.9, -0.9, 0.1,  0.1,  0.1, 1.1, 1.1, 1.1,-0.9, 0.1, 1.1,-0.9, 0.1, 1.1,-0.9, 0.1, 1.1]]
min_max = np.array(min_max)

tps = []
for i in range(18):
    tps.append(np.random.uniform(min_max[0,i], min_max[1,i]))
tps = [tps]

tps_Grid(tps)
tps_Grid(tps, norm=True)


### MRI training data

In [ ]:
columns = ['ImageA','ImageB']
for i in range(1,73):
    columns.append('t' + str(i))
    
train_mri_pd = pd.DataFrame(columns=columns)
test_mri_pd  = pd.DataFrame(columns=columns)

number_samples   = 0

file_names =  [i[20:] for i in sorted(glob.glob('./datasets/training/mri_*'))]

for i in range(10):
    count = 0
    for name in file_names: 
        new_row     = random_transform(transforms_pd, name)
        new_row_pd  = pd.DataFrame.from_dict(new_row)
        
        if count < 61:
            train_mri_pd = pd.concat((train_mri_pd, new_row_pd), ignore_index=True)
        else:
            test_mri_pd  = pd.concat((test_mri_pd, new_row_pd), ignore_index=True)
    
        number_samples += 1
        count          += 1

print(number_samples)
train_mri_pd.to_csv('training_data/tps/mri_train.csv', index=False)
test_mri_pd.to_csv('training_data/tps/mri_test.csv', index=False)
test_mri_pd


In [ ]:
min_max = [[-1.1, -1.1, -1.1, -0.1, -0.1, -0.1, 0.9, 0.9, 0.9,-1.1, -0.1, 0.9,-1.1, -0.1, 0.9,-1.1, -0.1, 0.9],
            [-0.9, -0.9, -0.9, 0.1,  0.1,  0.1, 1.1, 1.1, 1.1,-0.9, 0.1, 1.1,-0.9, 0.1, 1.1,-0.9, 0.1, 1.1]]

min_max = np.array(min_max)

columns = ['ImageA','ImageB']
for i in range(1,18):
    columns.append('t' + str(i))
    
train_mri_pd = pd.DataFrame(columns=columns)
test_mri_pd  = pd.DataFrame(columns=columns)

number_samples   = 0

file_names =  [i[20:] for i in sorted(glob.glob('./datasets/training/mri_*'))]

for i in range(10):
    count = 0
    for name in file_names: 
        new_row     = random_transform(image_name=name, min_max=min_max)
        new_row_pd  = pd.DataFrame.from_dict(new_row)
        
        if count < 61:
            train_mri_pd = pd.concat((train_mri_pd, new_row_pd), ignore_index=True)
        else:
            test_mri_pd  = pd.concat((test_mri_pd, new_row_pd), ignore_index=True)
    
        number_samples += 1
        count          += 1

print(number_samples)
train_mri_pd.to_csv('training_data/tps/mri_train.csv', index=False)
test_mri_pd.to_csv('training_data/tps/mri_test.csv', index=False)
test_mri_pd



### Create histo + mri csv

In [ ]:
def add_from_files(file_names, transforms_pd, train_pd, number_samples):
    count = 0
    
    for name in file_names: 
        new_row     = random_transform(transforms_pd, name)
        new_row_pd  = pd.DataFrame.from_dict(new_row)
        
        train_pd    = pd.concat((train_pd, new_row_pd), ignore_index=True)
        count       += 1
        
    #print('Count: ', count)
    number_samples += count
    
    return train_pd, number_samples
        
        
columns = ['ImageA','ImageB']
for i in range(1,73):
    columns.append('t' + str(i))

train_pd    = pd.DataFrame(columns=columns)

hist_names1    = [i[20:] for i in (glob.glob('./datasets/training/hist*'))]
hist_names2    = [i[20:] for i in (glob.glob('./datasets/training/HMU*'))]
mri_names1     = [i[20:] for i in (glob.glob('./datasets/training/mri_T*'))]
mri_names2     = [i[20:] for i in (glob.glob('./datasets/training/mri_H*'))]
dwi_names      = [i[20:] for i in (glob.glob('./datasets/training/dwi*'))]
b90_names      = [i[20:] for i in (glob.glob('./datasets/training/b90*'))]

print(len(hist_names1),len(hist_names2),len(mri_names1),len(mri_names2),len(dwi_names),len(b90_names))

number_samples   = 0
for i in range(10):
    random.shuffle(mri_names1)
    random.shuffle(mri_names2)
    
    train_pd, number_samples = add_from_files(hist_names1,      transforms_pd, train_pd, number_samples)
    train_pd, number_samples = add_from_files(hist_names2,      transforms_pd, train_pd, number_samples)
    train_pd, number_samples = add_from_files(mri_names1[:30],  transforms_pd, train_pd, number_samples)
    train_pd, number_samples = add_from_files(mri_names2[:30],  transforms_pd, train_pd, number_samples)
    train_pd, number_samples = add_from_files(dwi_names,        transforms_pd, train_pd, number_samples)
    train_pd, number_samples = add_from_files(b90_names,        transforms_pd, train_pd, number_samples)

print(number_samples)
train_pd.to_csv('training_data/tps/histo_mri_dwi_train.csv', index=False)


### Output transforms: histo-t2-dwi

In [ ]:
import matplotlib.pyplot    as plt
import numpy                as np
from process_img import * 
from geotnf.transformation import GeometricTnf
from skimage import io

def warp_image(source_image, transforms_1, transforms_2, affTnf, tpsTnf, tpsMRITnf):
    # Transforms
    theta_aff_1, theta_aff_2, theta_tps_1 = transforms_1
    theta_aff_3, theta_aff_4, theta_tps_2 = transforms_2
    
    # Warped with transform 1
    warped_image_1 = affTnf(source_image,     theta_aff_1.view(-1,2,3))
    warped_image_1 = affTnf(warped_image_1,   theta_aff_2.view(-1,2,3))
    warped_image_1 = tpsTnf(warped_image_1,   theta_tps_1)
    
    # Warped with transform 2
    warped_image_2 = affTnf(source_image,       theta_aff_3.view(-1,2,3))
    warped_image_2 = affTnf(warped_image_2,     theta_aff_4.view(-1,2,3))
    warped_image_2 = tpsMRITnf(warped_image_2,  theta_tps_2)
    
    # Warped with transform 1+2
    warped_image_total = affTnf(warped_image_1,         theta_aff_3.view(-1,2,3))
    warped_image_total = affTnf(warped_image_total,     theta_aff_4.view(-1,2,3))
    warped_image_total = tpsMRITnf(warped_image_total,  theta_tps_2)
    
    # Un-normalize image and convert to numpy
    warped_image_1_np = normalize_image(warped_image_1,forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
    warped_image_2_np = normalize_image(warped_image_2,forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
    warped_image_total_np = normalize_image(warped_image_total,forward=False).data.squeeze(0).transpose(0,1).transpose(1,2).cpu().numpy()
        
    # Ignore negative values
    warped_image_1_np[warped_image_1_np < 0] = 0
    warped_image_2_np[warped_image_2_np < 0] = 0    
    warped_image_total_np[warped_image_total_np < 0] = 0 
    
    return warped_image_1_np, warped_image_2_np, warped_image_total_np
    
    
def tps_Grid_full(transforms_1, transforms_2, histo_image, out_size=500, use_cuda=False):
    """
    Apply transforms to the sample grid
    """
    
    # Load geometric models
    affTnf      = GeometricTnf(geometric_model='affine', out_h=out_size, out_w=out_size, use_cuda=use_cuda)
    tpsTnf      = GeometricTnf(geometric_model='tps',    out_h=out_size, out_w=out_size, use_cuda=use_cuda)
    tpsMRITnf   = GeometricTnf(geometric_model='tps-mri',out_h=out_size, out_w=out_size, use_cuda=use_cuda)
    
    # Preprocess image 
    grid_image = io.imread('../Dataset/Data/grid.png')
    grid_image = np.squeeze(grid_image[:,:,:3])
    grid_image[grid_image<250] = 0
    
    grid_image  = process_image(grid_image, use_cuda=False, high_res=True)
    histo_image = process_image(histo_image,  use_cuda=False, high_res=True)

    grid_warped_1,  grid_warped_2,  grid_warped_total  = warp_image(grid_image,  transforms_1, transforms_2, affTnf, tpsTnf, tpsMRITnf)
    histo_warped_1, histo_warped_2, histo_warped_total = warp_image(histo_image, transforms_1, transforms_2, affTnf, tpsTnf, tpsMRITnf)

    grid_image  = grid_image.permute(0,2,3,1)
    histo_image = histo_image.permute(0,2,3,1)
    
    fig, axs = plt.subplots(2,4, figsize=(15, 6))
    axs[0,0].set_title('Source')
    axs[0,0].imshow(np.squeeze(grid_image))
    axs[1,0].imshow(np.squeeze(histo_image))
    
    axs[0,1].set_title('Transform histo-T2')
    axs[0,1].imshow(grid_warped_1)
    axs[1,1].imshow(histo_warped_1)
    
    axs[0,2].set_title('Transform T2-DWI')
    axs[0,2].imshow(grid_warped_2)
    axs[1,2].imshow(histo_warped_2)
    
    axs[0,3].set_title('Transform histo-DWI')
    axs[0,3].imshow(grid_warped_total)
    axs[1,3].imshow(histo_warped_total)
    

In [ ]:
from transform_histo_dwi import get_data, create_tensor
from glob import glob

sid = 'HMU_010_FH'
json_path_histo_t2  = './transforms/transform_' + sid + '.json'
json_path_t2_dwi    = './transforms/transform_' + sid + '_T2_DWI.json'
img_path   = os.path.join('./results/preprocess/hist' , sid+'_high_res/')

json_data_histo_t2 = get_data(json_path_histo_t2)
json_data_t2_dwi   = get_data(json_path_t2_dwi)

# Get image paths
img_path   = os.path.join('./results/preprocess/hist' , sid)
all_paths = sorted(glob(img_path + '/hist*.png'))

# Get transformation params
for i in range(4):
    theta_aff_1 = json_data_histo_t2[str(i)]["affine_1"]
    theta_aff_2 = json_data_histo_t2[str(i)]["affine_2"]
    theta_tps_1 = json_data_histo_t2[str(i)]["tps"]
    transforms_1  = (create_tensor(theta_aff_1, use_cuda=False), create_tensor(theta_aff_2, use_cuda=False), create_tensor(theta_tps_1, use_cuda=False)) 

    theta_aff_3 = json_data_t2_dwi[str(i)]["affine_1"]
    theta_aff_4 = json_data_t2_dwi[str(i)]["affine_2"]
    theta_tps_2 = json_data_t2_dwi[str(i)]["tps"]
    transforms_2  = (create_tensor(theta_aff_3, use_cuda=False), create_tensor(theta_aff_4, use_cuda=False), create_tensor(theta_tps_2, use_cuda=False))
    
    source_img = io.imread(all_paths[i])

    tps_Grid_full(transforms_1, transforms_2, source_img)